### Matching DoG keypoints between original and rotated image for OpenCV, Kornia and VLFeat

* measuring the reprojection mean error and mse
* matching based on the position (original image) and reprojected position (scaled image)     
* matching a) based on mutual nearest neighbor b) with reprojection error < threshold (usually 2 pixels)
* optionally filtered with "size check" 
  * ratio = (kpt_size_scaled / scale) / kpt_size_original
  * 1 + abs(1 - ratio) < threshold (usually 1.1)
  * doesn't work for Kornia as the sizes are not consistent there 
* rotated by multiples of 90 degrees
* measured on the first n images from the marketplace_dataset (scene ai_001_001) 
from the ml-hypersim dataset (https://github.com/apple/ml-hypersim)


In [1]:
import os

from dataset_utils import *
from geometry import mnn
from tqdm.notebook import tqdm


def write_results(means, mses, name, correct=True):

    correct_list = [0, 1] if correct else [0]
    for corrected in correct_list:
        title = f"{name}:" if corrected == 0 else f"{name} corrected:"
        print()
        print(title)
        print()
        print("                 Mean error")
        print("Rotation          x        y")
        for i, mean in enumerate(means[corrected]):
            sp = " " if i == 0 else ""
            print(f" {sp}{90 + i * 90} deg     {mean[0]:+.03f}   {mean[1]:+.03f}")
        print()
        print("Rotation        Mse")
        for i, mse in enumerate(mses[corrected]):
            sp = " " if i == 0 else ""
            print(f" {sp}{90 + i * 90} deg      {mse:.03f}")
        print()


def detect_kpts(img_np, detector, correct):

    kpts = detector.detect(img_np, mask=None)

    if len(kpts) == 0:
        return [], []

    kpt_f = torch.tensor([[kp.pt[0], kp.pt[1]] for kp in kpts])
    if correct:
        kpt_f -= 0.25

    return kpt_f


def back_project_kpts(kpts, img_rotated, rotations):
    kpts_h = torch.ones((kpts.shape[0], 3)).to(dtype=kpts.dtype)
    kpts_h[:, :2] = kpts.clone()
    Hs = torch.from_numpy(rotation_gt_Hs(img_rotated)[3 - rotations]).to(dtype=kpts.dtype)
    kpts = (Hs @ kpts_h.T)[:2].T
    return kpts


def rotate_experiment_mnn(detector, img_to_show, err_th, correct=True):

    img_dir = "../imgs/hypersim"
    files = sorted(["{}/{}".format(img_dir, fn) for fn in os.listdir(img_dir)][:img_to_show])
    means = np.zeros((2, 3, 2))
    mses = np.zeros((2, 3))
    correct_list = [0, 1] if correct else [0]
    for rotation in tqdm(range(1, 4), leave=False):
        for corrected in correct_list:
            errors = torch.zeros((0, 2))
            for file_path in files:

                img_np_o = np.array(Image.open(file_path))
                kpts_0 = detect_kpts(img_np_o, detector, correct=corrected==1)

                img_np_r = np.rot90(img_np_o, rotation, [0, 1])
                kpts_1 = detect_kpts(img_np_r, detector, correct=corrected==1)

                kpts_1 = back_project_kpts(kpts_1, img_np_r, rotation)
                kpts_0, kpts_1, _, _ = mnn(kpts_0, kpts_1, err_th=err_th)

                errors = torch.vstack((errors, kpts_1 - kpts_0))
            means[corrected, rotation - 1] = errors.mean(dim=0)
            mses[corrected, rotation - 1] = (torch.linalg.norm(errors, axis=1) ** 2).mean()

    return means, mses

In [2]:
import cv2 as cv
detector = cv.SIFT_create()

means, mses = rotate_experiment_mnn(detector, img_to_show=100, err_th=2)
write_results(means, mses, "OpenCV DoG")

  0%|          | 0/3 [00:00<?, ?it/s]


OpenCV DoG:

                 Mean error
Rotation          x        y
  90 deg     -0.492   -0.001
 180 deg     -0.492   -0.491
 270 deg     -0.001   -0.494

Rotation        Mse
  90 deg      0.269
 180 deg      0.526
 270 deg      0.269


OpenCV DoG corrected:

                 Mean error
Rotation          x        y
  90 deg     +0.002   -0.001
 180 deg     +0.000   +0.001
 270 deg     -0.001   +0.001

Rotation        Mse
  90 deg      0.021
 180 deg      0.034
 270 deg      0.022



In [3]:
from kornia.feature.integrated import SIFTFeature
from kornia.utils import image_to_tensor
import torch


class NumpyKorniaSiftDetector:

    def __init__(self, device=torch.device("cpu")):
        self.device = device
        self.sf = SIFTFeature(device=device)

    @staticmethod
    def cv_kpt_from_laffs_responses(laffs, responses):
        kpts = []
        for i, response in enumerate(responses[0]):
            yx = laffs[0, i, :, 2]
            kp = cv.KeyPoint(yx[0].item(), yx[1].item(), response.item(), angle=0)
            kpts.append(kp)
        return kpts

    def detect(self, img, mask):
        assert mask is None, "not implemented with non-trivial mask"
        if len(img.shape) == 2:
            img = img[:, :, None]
        else:
            img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        img_t = (image_to_tensor(img, False).float() / 255.).to(device=self.device)
        laffs, responses, descs = self.sf(img_t, mask=None)
        kpts = self.cv_kpt_from_laffs_responses(laffs, responses)
        return kpts


detector = NumpyKorniaSiftDetector()

means, mses = rotate_experiment_mnn(detector, img_to_show=2, err_th=2)
write_results(means, mses, "Kornia DoG")

  0%|          | 0/3 [00:00<?, ?it/s]


Kornia DoG:

                 Mean error
Rotation          x        y
  90 deg     -0.476   -0.001
 180 deg     -0.463   -0.462
 270 deg     -0.004   -0.471

Rotation        Mse
  90 deg      0.429
 180 deg      0.672
 270 deg      0.423


Kornia DoG corrected:

                 Mean error
Rotation          x        y
  90 deg     +0.001   +0.001
 180 deg     +0.006   +0.006
 270 deg     -0.002   +0.004

Rotation        Mse
  90 deg      0.205
 180 deg      0.229
 270 deg      0.202



In [4]:
import cv2 as cv
from hloc.extractors.dog import DoG
from kornia.utils import image_to_tensor

class HlocSiftDetector:

    sift_conf = {
        'options': {
            'first_octave': -1,
            'peak_threshold': 0.01,
        },
        'descriptor': 'sift',
        'max_keypoints': -1,
        'patch_size': 32,
        'mr_size': 12,
    }

    def __init__(self, conf=sift_conf, device=torch.device("cpu")):
        self.dog = DoG(conf)
        self.device = device

    def create_cv_kpts(self, keypoints, scales, oris):
        keypoints = keypoints[0]
        kpts = []
        for i, kpt in enumerate(keypoints):
            x = kpt[0].item()
            y = kpt[1].item()
            size = scales[0, i].item()
            angle = oris[0, i].item()
            kp = cv.KeyPoint(x, y, size=size, angle=angle)
            kpts.append(kp)
        return kpts

    def detect(self, img, mask):
        assert mask is None, "not implemented with non-trivial mask"
        if len(img.shape) == 2:
            img = img[:, :, None]
        else:
            img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        img_t = (image_to_tensor(img, False).float() / 255.).to(device=self.device)
        ret_dict = self.dog({"image": img_t})
        cv_kpts = self.create_cv_kpts(ret_dict['keypoints'], ret_dict['scales'], ret_dict['oris'])
        return cv_kpts


detector = HlocSiftDetector()

means, mses = rotate_experiment_mnn(detector, img_to_show=2, err_th=2, correct=False)
write_results(means, mses, "VLFeat DoG", correct=False)


  0%|          | 0/3 [00:00<?, ?it/s]


VLFeat DoG:

                 Mean error
Rotation          x        y
  90 deg     -0.001   -0.001
 180 deg     -0.001   -0.003
 270 deg     -0.002   -0.003

Rotation        Mse
  90 deg      0.006
 180 deg      0.011
 270 deg      0.006

